In [1]:
#| echo: false
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Math

# a)

# Legger inn data
df = pd.read_excel("Assignment_1_data.xlsx")

OSEBX = df['OSEBX'].to_numpy()      # N x 1 vektor
Equinor = df['EQUINOR'].to_numpy()  # N x 1 vektor

# Setter sammen til en N x 2 matrise
Y = np.column_stack((OSEBX, Equinor))

# Definerer vektoren (N x 1)
N = Y.shape[0]
i = np.ones((N, 1))

# Beregner gjennomsnitt med vektorformelen (1/N * i'Y)
mean_returns_daily = (1/N) * (i.T @ Y)

print("Gjennomsnittlig daglig avkastning (OSEBX, Equinor):", mean_returns_daily)

# Multiplisere med antall handelsdager
mean_returns_annual = mean_returns_daily * 252

print("Gjennomsnittlig årlig avkastning (OSEBX, Equinor):", mean_returns_annual)

Gjennomsnittlig daglig avkastning (OSEBX, Equinor): [[0.00032235 0.00040407]]
Gjennomsnittlig årlig avkastning (OSEBX, Equinor): [[0.08123316 0.10182591]]


In [2]:
# b)

mean_matrix = i @ mean_returns_daily          # shape: (N, 2)

# Trekke ut hver mean-vektor (N×1)
meanvec_osebx = mean_matrix[:, [0]]
meanvec_eqnr  = mean_matrix[:, [1]]

print("Vektor for gjennomsnitt (OSEBX):",meanvec_osebx)
print("Vektor for gjennomsnitt (Equinor):",meanvec_eqnr)


Vektor for gjennomsnitt (OSEBX): [[0.00032235]
 [0.00032235]
 [0.00032235]
 ...
 [0.00032235]
 [0.00032235]
 [0.00032235]]
Vektor for gjennomsnitt (Equinor): [[0.00040407]
 [0.00040407]
 [0.00040407]
 ...
 [0.00040407]
 [0.00040407]
 [0.00040407]]


The expression: $$ y - \dfrac{1}{N}ii'y $$ represents the vector of deviations from the mean. Here, $y$ is the vector of returns, and $\tfrac{1}{N}ii'y$ is the mean return replicated into an $N \times 1$ vector. Subtracting this from $y$ gives a vector of demeaned data, showing how each return differs from the average. This step is essential for calculating variance and covariance.  

In [3]:
# c)

#| echo: false
deviation = Y - mean_matrix
print(deviation)
total_sum = np.sum(deviation)

display(Markdown(
f"""
\n
Subtracting the mean from each entry gives the deviation from mean, useful when calculating variance and covariance.\n
The sum of all deviations is  {total_sum}, which is $\\approx$ 0 as expected. \n
"""
))

[[ 0.00648285  0.01398467]
 [-0.00222521  0.01378056]
 [-0.00942367 -0.01458871]
 ...
 [-0.00066532 -0.01313964]
 [-0.02596501 -0.01856638]
 [ 0.01971299  0.01993748]]





Subtracting the mean from each entry gives the deviation from mean, useful when calculating variance and covariance.

The sum of all deviations is  -1.7208456881689926e-15, which is $\approx$ 0 as expected. 



d)

The matrix is defined as  

$$
M_0 = I - \dfrac{1}{N}ii'
$$

The matrix $M_0$ is **symmetric** because  

$$
M_0' = \left(I - \dfrac{1}{N}ii'\right)' 
= I' - \dfrac{1}{N}(ii')' 
= I - \dfrac{1}{N}ii' 
= M_0
$$

The matrix $M_0$ is **idempotent** because  

$$
M_0^2 = \left(I - \dfrac{1}{N}ii'\right)\left(I - \dfrac{1}{N}ii'\right) 
= I - \dfrac{2}{N}ii' + \dfrac{1}{N^2}(ii'ii')
$$

Since  

$$
ii'i = i(i'i) = iN
$$  

it follows that  

$$
ii'ii' = (iN)i' = N(ii')
$$

Therefore,  

$$
M_0^2 = I - \dfrac{2}{N}ii' + \dfrac{1}{N}(ii') 
= I - \dfrac{1}{N}ii' 
= M_0
$$

Thus, $M_0$ is both symmetric and idempotent. It acts as a **projection matrix** that removes the mean from a vector $y$, leaving only the deviations from the average.


In [5]:
#| echo: false
# e) sjekk for begge serier samtidig

N = len(OSEBX)
i_vec = np.ones(N)
M0 = np.eye(N) - (1/N) * np.outer(i_vec, i_vec)

# Bruk begge serier som en N×2 matrise
Y = np.column_stack((OSEBX, Equinor))

# Left-hand side: sum of squared deviations for hver kolonne
lhs = np.sum((Y - mean_matrix)**2, axis=0)

# Right-hand side: y'M0y for hver kolonne
rhs = np.diag(Y.T @ M0 @ Y)

display(Markdown(
f"""
Using the equation:

$$
\\sum_t (y_t - \\bar{{y}})^2 = y'M_0y
$$

Numerical check (both series):

- OSEBX: LHS = {lhs[0]:.6f}, RHS = {rhs[0]:.6f}  
- Equinor: LHS = {lhs[1]:.6f}, RHS = {rhs[1]:.6f}  
"""
))


Using the equation:

$$
\sum_t (y_t - \bar{y})^2 = y'M_0y
$$

Numerical check (both series):

- OSEBX: LHS = 0.917406, RHS = 0.917406  
- Equinor: LHS = 1.553563, RHS = 1.553563  


f)   We define

$$
M_0 \;=\; I - \frac{1}{N}\, i i',
$$
where $i = (1,1,\dots,1)'\in\mathbb{R}^N$.


$$
(M_0)_{ts} =
\begin{cases}
1 - \dfrac{1}{N}, & \text{if } t=s,\\[6pt]
-\dfrac{1}{N}, & \text{if } t\neq s.
\end{cases}
$$

The matrix $M^0$ has diagonal entries equal to $1-\dfrac{1}{N}$ and an off-diagonal entries equal to $-\dfrac{1}{N}$
$$
M_0 \;=\;
\begin{bmatrix}
1-\tfrac{1}{N} & -\tfrac{1}{N} & \cdots & -\tfrac{1}{N}\\
-\tfrac{1}{N} & 1-\tfrac{1}{N} & \cdots & -\tfrac{1}{N}\\
\vdots & \vdots & \ddots & \vdots\\
-\tfrac{1}{N} & -\tfrac{1}{N} & \cdots & 1-\tfrac{1}{N}
\end{bmatrix}.
$$
hvor $1-\tfrac{1}{N} \approx 0.99977$ og $-\tfrac{1}{N} \approx -0.00023$


In [19]:
# g)

std_daily_OSEBX  = OSEBX.std()
std_annual_OSEBX = std_daily_OSEBX * np.sqrt(252)

print(f"1. Standard deviation OSEBX (daily):  {std_daily_OSEBX:.6f}")
print(f"2. Standard deviation OSEBX (annual): {std_annual_OSEBX:.6f}\n")


1. Standard deviation OSEBX (daily):  0.014433
2. Standard deviation OSEBX (annual): 0.229117



In [16]:
# h)

std_daily_Equinor  = Equinor.std()
std_annual_Equinor = std_daily_Equinor * np.sqrt(252)

print(f"1. Standard deviation Equinor (daily):  {std_daily_Equinor:.6f}")
print(f"2. Standard deviation Equinor (annual): {std_annual_Equinor:.6f}\n")


1. Standard deviation Equinor (daily):  0.018782
2. Standard deviation Equinor (annual): 0.298154



In [17]:
# i)

cov_OSEBX_EQNR = np.cov(OSEBX, Equinor)[0, 1]

print(f"Covariance between OSEBX and Equinor: {cov_OSEBX_EQNR:.6f}\n")


Covariance between OSEBX and Equinor: 0.000211



In [18]:
# j)
 
beta_Equinor = cov_OSEBX_EQNR / (std_daily_OSEBX**2)

print(f"Beta for Equinor (relative to OSEBX): {beta_Equinor:.6f}\n")


Beta for Equinor (relative to OSEBX): 1.012217

